In [2]:
from google.colab import drive
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [85]:
import pandas as pd
import numpy as np
from datetime import datetime
import holidays
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score,classification_report,confusion_matrix
import matplotlib.pyplot as plt
from scipy.stats import  randint
import math
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import pickle

## Build X_train

In [35]:
X_train = pd.read_csv('fl01_train')

In [4]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 991894 entries, 0 to 991893
Data columns (total 22 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Unnamed: 0           991894 non-null  int64  
 1   Unnamed: 0.1         991894 non-null  int64  
 2   fl_date              991894 non-null  object 
 3   op_unique_carrier    991894 non-null  object 
 4   tail_num             988128 non-null  object 
 5   op_carrier_fl_num    991894 non-null  int64  
 6   origin_airport_id    991894 non-null  int64  
 7   dest_airport_id      991894 non-null  int64  
 8   crs_dep_time         991894 non-null  int64  
 9   dep_delay            976157 non-null  float64
 10  crs_arr_time         991894 non-null  int64  
 11  arr_delay            974040 non-null  float64
 12  cancelled            991894 non-null  float64
 13  cancellation_code    15724 non-null   object 
 14  crs_elapsed_time     991888 non-null  float64
 15  distance         

In [36]:
X_train = X_train[['fl_date','op_unique_carrier','op_carrier_fl_num','origin_airport_id','crs_dep_time','dest_airport_id','crs_arr_time','distance','arr_delay','carrier_delay','weather_delay','nas_delay','security_delay','late_aircraft_delay']]

In [51]:
X_train.head()

,distance,uni_fl,arr_airport_hour,dep_airport_hour,is_holiday,dow_sin,dow_cos
0,2106.0,-8.642857,-6.284264,0.133843,0,-0.974927,-0.222525
1,1243.0,-1.666667,-4.724080,8.300518,0,-0.781834,0.623486
2,612.0,-1.461538,10.879245,15.880258,0,-0.000005,1.000000
3,515.0,-5.575758,2.895881,14.562380,0,-0.974927,-0.222525
4,1561.0,-7.333333,2.227273,-0.149701,0,-0.000005,1.000000


In [40]:
y_del_class = X_train['delay_class']

In [42]:
X_train['delay_class'].to_csv('train_y_del_cls')

In [38]:
def delay_cla(a,b,c,d,e):
    l = pd.Series([a,b,c,d,e]).fillna(0)
    if l.sum() == 0:
        return 0
    else: 
        return l.idxmax() + 1

#print(delay_cla(None,1,None,3,None))
X_train['delay_class'] = X_train[['carrier_delay','weather_delay','nas_delay','security_delay','late_aircraft_delay']].apply(lambda X_train: delay_cla(X_train['carrier_delay'],X_train['weather_delay'],X_train['nas_delay'],X_train['security_delay'],X_train['late_aircraft_delay']),axis=1)
#X_train['delay_class'] = np.vectorize(delay_cla)(X_train[['carrier_delay','weather_delay','nas_delay','security_delay','late_aircraft_delay']])
#X_train['delay_class'] = np.vectorize(delay_cla)(X_train['carrier_delay'],X_train['weather_delay'],X_train['nas_delay'],X_train['security_delay'],X_train['late_aircraft_delay'])

In [50]:
X_train.drop(columns=['carrier_delay','weather_delay','nas_delay','security_delay','late_aircraft_delay'],inplace=True)

In [16]:
arr_d_match = X_train.groupby(['delay_cls'])['arr_delay'].mean()

In [17]:
arr_d_match

delay_cls
0    -84.188470
1    -31.147642
2    -11.231984
3      8.376291
4     35.723876
5    100.855927
6    320.689599
Name: arr_delay, dtype: float64

In [43]:
# after test data, arr_delay should be 0. 
X_train['arr_delay'].fillna(0,inplace=True)

#remove outliner
#X_train = X_train[(X_train['arr_delay'] > -80) & (X_train['arr_delay'] <= 480)] 

#combine features
def combine_str_num(x,y):
    return x + str(y)
X_train['uni_fl'] = np.vectorize(combine_str_num)(X_train['op_unique_carrier'],X_train['op_carrier_fl_num'])
X_train.drop(columns=['op_unique_carrier','op_carrier_fl_num'],inplace=True)

# feature coding
total_target_mean = np.mean(X_train['arr_delay'])
unifl_target_mean = X_train.groupby(['uni_fl'])['arr_delay'].mean()
X_train['uni_fl'] =X_train['uni_fl'].map(unifl_target_mean)

In [44]:
## Combine the airportid and related hour
def airport_hour(id,t):
    '''
    combine airport and hour to a new field
    '''
    h = str(t)[:-2]
    if len(h) < 1:
        return str(id) + '24'
    elif len(h) < 2:
        return str(id) + '0' + h
    else:
        return str(id) + h
    
X_train['arr_airport_hour'] = np.vectorize(airport_hour)(X_train['dest_airport_id'],X_train['crs_arr_time'])
X_train['dep_airport_hour'] = np.vectorize(airport_hour)(X_train['origin_airport_id'],X_train['crs_dep_time'])

# feature coding
dep_hour_target_mean = X_train.groupby(['dep_airport_hour'])['arr_delay'].mean()
X_train['dep_airport_hour'] =X_train['dep_airport_hour'].map(dep_hour_target_mean)
arr_hour_target_mean = X_train.groupby(['arr_airport_hour'])['arr_delay'].mean()
X_train['arr_airport_hour'] =X_train['arr_airport_hour'].map(arr_hour_target_mean)

X_train.drop(columns=['dest_airport_id','crs_arr_time','origin_airport_id','crs_dep_time'],inplace=True)

In [45]:
#transform fl_date
#def getmonth(x):
#    return x.split(sep='-')[1]
#X_train['month'] = X_train['fl_date'].apply(getmonth)

def getdayofweek(x):
    year_s, mon_s, day_s = x.split('-')
    fl_d = datetime(int(year_s), int(mon_s), int(day_s))
    return fl_d.weekday() + 1
X_train['day_of_week'] = X_train['fl_date'].apply(getdayofweek)

us_holidays = holidays.UnitedStates()
def isholiday(x):
    year_s, mon_s, day_s = x.split('-')
    if datetime(int(year_s), int(mon_s), int(day_s)) in us_holidays:
        return 1
    else:
        return 0
X_train['is_holiday'] = X_train['fl_date'].apply(isholiday)


pi=3.14159
def transformation(column):
  max_value = column.max()
  sin_values = [math.sin((2*pi*x)/max_value) for x in list(column)]
  cos_values = [math.cos((2*pi*x)/max_value) for x in list(column)]
  return sin_values, cos_values

sin_l, cos_l = transformation(X_train['day_of_week'])

X_train['dow_sin'] = sin_l
X_train['dow_cos'] = cos_l

In [47]:
y_train = X_train['delay_class']

X_train = X_train.drop(columns=['fl_date','arr_delay','day_of_week','delay_class'])

#X_train = pd.get_dummies(X_train,columns=['month','day_of_week'])

In [18]:
#scaler = StandardScaler()
#X = scaler.fit_transform(X_train)

In [ ]:
#x_small = X[:100000]
#y_small = y_train[:100000]

### Build X_test

In [57]:
X_test = pd.read_csv('fl01_test')

In [58]:
X_test = X_test[['fl_date','op_unique_carrier','op_carrier_fl_num','origin_airport_id','crs_dep_time','dest_airport_id','crs_arr_time','distance','arr_delay','carrier_delay','weather_delay','nas_delay','security_delay','late_aircraft_delay']]

In [61]:
X_test.head()

,fl_date,op_unique_carrier,op_carrier_fl_num,origin_airport_id,crs_dep_time,dest_airport_id,crs_arr_time,distance,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,delay_class
0,2018-01-21,AA,9,12478,700,14771,1049,2586.0,-36.0,NaN,NaN,NaN,NaN,NaN,0
1,2018-01-12,WN,1832,11278,600,13204,825,759.0,82.0,81.0,0.0,1.0,0.0,0.0,1
2,2019-01-27,WN,1699,14893,825,10423,1345,1481.0,-24.0,NaN,NaN,NaN,NaN,NaN,0
3,2018-01-01,EV,3978,13930,715,14524,1018,642.0,6.0,NaN,NaN,NaN,NaN,NaN,0
4,2018-01-22,F9,649,12953,920,11193,1133,585.0,19.0,0.0,0.0,19.0,0.0,0.0,3


In [71]:
X_test.isnull().sum()

distance                    0
carrier_delay          202740
weather_delay          202740
nas_delay              202740
security_delay         202740
late_aircraft_delay    202740
uni_fl                      0
arr_airport_hour            0
dep_airport_hour            0
is_holiday                  0
dow_sin                     0
dow_cos                     0
dtype: int64

In [60]:
X_test['delay_class'] = np.vectorize(delay_cla)(X_test['carrier_delay'],X_test['weather_delay'],X_test['nas_delay'],X_test['security_delay'],X_test['late_aircraft_delay'])

In [64]:
y_test_del_class = X_test['delay_class']
X_test['delay_class'].to_csv('test_y_del_cls')

In [72]:
X_test.drop(columns=['carrier_delay','weather_delay','nas_delay','security_delay','late_aircraft_delay'],inplace=True)

In [65]:
# after test data, arr_delay should be 0. 
X_test['arr_delay'].fillna(0,inplace=True)

#remove outliner
#X_train = X_train[(X_train['arr_delay'] > -80) & (X_train['arr_delay'] < 180)] 

#combine features
def combine_str_num(x,y):
    return x + str(y)
X_test['uni_fl'] = np.vectorize(combine_str_num)(X_test['op_unique_carrier'],X_test['op_carrier_fl_num'])
X_test.drop(columns=['op_unique_carrier','op_carrier_fl_num'],inplace=True)

# feature coding
#unifl_target_mean = X_test.groupby(['uni_fl'])['arr_delay'].mean()
X_test['uni_fl'] =X_test['uni_fl'].map(unifl_target_mean)
X_test['uni_fl'].fillna(total_target_mean,inplace=True)

In [ ]:
#Transform arr delay
#X_test['delay_cls'] = X_test['arr_delay'].apply(tran_delay)

In [67]:
## Combine the airportid and related hour
def airport_hour(id,t):
    '''
    combine airport and hour to a new field
    '''
    h = str(t)[:-2]
    if len(h) < 1:
        return str(id) + '24'
    elif len(h) < 2:
        return str(id) + '0' + h
    else:
        return str(id) + h
    
X_test['arr_airport_hour'] = np.vectorize(airport_hour)(X_test['dest_airport_id'],X_test['crs_arr_time'])
X_test['dep_airport_hour'] = np.vectorize(airport_hour)(X_test['origin_airport_id'],X_test['crs_dep_time'])

# feature coding
#dep_hour_target_mean = X_train.groupby(['dep_airport_hour'])['arr_delay'].mean()
X_test['dep_airport_hour'] =X_test['dep_airport_hour'].map(dep_hour_target_mean)
#arr_hour_target_mean = X_train.groupby(['arr_airport_hour'])['arr_delay'].mean()
X_test['arr_airport_hour'] =X_test['arr_airport_hour'].map(arr_hour_target_mean)

X_test['arr_airport_hour'].fillna(total_target_mean,inplace=True)
X_test['dep_airport_hour'].fillna(total_target_mean,inplace=True)

X_test.drop(columns=['dest_airport_id','crs_arr_time','origin_airport_id','crs_dep_time'],inplace=True)

In [68]:
#transform fl_date
#def getmonth(x):
#    return x.split(sep='-')[1]
#X_test['month'] = X_test['fl_date'].apply(getmonth)

def getdayofweek(x):
    year_s, mon_s, day_s = x.split('-')
    fl_d = datetime(int(year_s), int(mon_s), int(day_s))
    return fl_d.weekday() + 1
X_test['day_of_week'] = X_test['fl_date'].apply(getdayofweek)

us_holidays = holidays.UnitedStates()
def isholiday(x):
    year_s, mon_s, day_s = x.split('-')
    if datetime(int(year_s), int(mon_s), int(day_s)) in us_holidays:
        return 1
    else:
        return 0
X_test['is_holiday'] = X_test['fl_date'].apply(isholiday)



pi=3.14159
def transformation(column):
  max_value = column.max()
  sin_values = [math.sin((2*pi*x)/max_value) for x in list(column)]
  cos_values = [math.cos((2*pi*x)/max_value) for x in list(column)]
  return sin_values, cos_values

sin_l, cos_l = transformation(X_test['day_of_week'])

X_test['dow_sin'] = sin_l
X_test['dow_cos'] = cos_l



In [69]:
y_test = X_test['delay_class']

X_test = X_test.drop(columns=['fl_date','arr_delay','day_of_week','delay_class'])

#X_test = pd.get_dummies(X_test,columns=['month','day_of_week'])

#X_test = scaler.transform(X_test)

In [14]:
###  Decision Tree

In [ ]:
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [73]:
y_pred = clf.predict(X_test)

In [74]:
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",accuracy_score(y_test, y_pred))
#print("ROC score", roc_auc_score(y_test,y_pred,multi_class='ovr'))

Accuracy: 0.7696694008242799


In [77]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.83      0.93      0.87    202740
           1       0.08      0.04      0.05     12054
           2       0.03      0.01      0.01      1756
           3       0.16      0.07      0.10     14254
           4       0.00      0.00      0.00        66
           5       0.16      0.07      0.10     17104

    accuracy                           0.77    247974
   macro avg       0.21      0.19      0.19    247974
weighted avg       0.70      0.77      0.73    247974



In [72]:
#Random Forest

In [47]:
#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

y_pred_r=clf.predict(X_test)

In [52]:
clf.feature_importances_

array([0.20215096, 0.26113699, 0.21012088, 0.21025372, 0.00632843,
       0.05484196, 0.05516707])

In [78]:
param_grid = {'n_estimators':[50], 'max_depth':[6,8,10],'min_samples_leaf':[2,6,10]}
rg = GridSearchCV(estimator=RandomForestClassifier(), param_grid=param_grid, cv=5, n_jobs=-1)

In [79]:
rg.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [6, 8, 10],
                         'min_samples_leaf': [2, 6, 10], 'n_estimators': [50]})

In [81]:
rg.best_estimator_

RandomForestClassifier(max_depth=10, min_samples_leaf=2, n_estimators=50)

In [80]:
y_pred_r = rg.predict(X_test)

In [82]:
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",accuracy_score(y_test, y_pred_r))

Accuracy: 0.8176139433972917


In [84]:
print(classification_report(y_test,y_pred_r,zero_division=))

              precision    recall  f1-score   support

           0       0.82      1.00      0.90    202740
           1       0.00      0.00      0.00     12054
           2       0.00      0.00      0.00      1756
           3       0.53      0.00      0.00     14254
           4       0.00      0.00      0.00        66
           5       0.50      0.00      0.00     17104

    accuracy                           0.82    247974
   macro avg       0.31      0.17      0.15    247974
weighted avg       0.73      0.82      0.74    247974



/home/ec2-user/anaconda3/envs/pytorch_latest_p37/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [86]:
confusion_matrix(y_test,y_pred_r)

array([[202734,      0,      0,      5,      0,      1],
       [ 12054,      0,      0,      0,      0,      0],
       [  1756,      0,      0,      0,      0,      0],
       [ 14242,      0,      0,     10,      0,      2],
       [    66,      0,      0,      0,      0,      0],
       [ 17097,      0,      0,      4,      0,      3]])

In [105]:
pickle.dump(rg,open('Grid_RandomForest_1.sav','wb'))

In [30]:
y_pred = loaded_model.predict(X_test)

In [54]:
print("Accuracy:",accuracy_score(y_test, y_pred_r))

Accuracy: 0.4876559639317025


In [62]:
y_pred_r

array([1, 2, 2, ..., 2, 2, 2])

In [56]:
print(unique,counts)

[0 1 2 3 4 5 6] [    62  16688 180105  30318  12613   6798   1390]


In [55]:
unique, counts = np.unique(y_pred_r, return_counts=True)

In [60]:
y_arr = np.array([arr_d_match[i] for i in y_pred_r])

In [61]:
y_arr

array([-31.1476422 , -11.23198414, -11.23198414, ..., -11.23198414,
       -11.23198414, -11.23198414])

In [63]:
arr_d_match

delay_cls
0    -84.188470
1    -31.147642
2    -11.231984
3      8.376291
4     35.723876
5    100.855927
6    320.689599
Name: arr_delay, dtype: float64

In [67]:
y_arr = np.array([arr_d_match[i] for i in y_pred])
r2_score(Y_compare,y_arr)

-0.09276152459671638

In [64]:
## four round  48.09246
r2_score(y_test,y_pred)

0.025932099879509285

In [94]:
## third round  48.67
r2_score(y_test,y_pred)

0.03500066824855497

In [21]:
## second round with arr_delay <480
#r2_score(y_test, y_pred)

0.02144653066658464

In [27]:
## first round with arr_delay < 180
#

0.013895135285908466

In [ ]:
## first try on XGBoost 
#r2_score(y_test,y_pred)

-0.0005833909885790689